In [1]:
import pandas as pd

In [2]:
lang = 'es'
#multicardioner_track2_cardioccc_dev_es_predictions_joined
mentions_file = f'output/multicardioner_track2_cardioccc_dev_{lang}_predictions_joined.tsv'
#multicardioner_track2_cardioccc_dev_en_predictions_joined
external_mentions_file = f'output/extracted_drugs_cardioccc_dev_{lang}.tsv'
result_file = f'output/multicardioner_track2_cardioccc_dev_{lang}_predictions_xlmr_medical_True_joined_combined_wo_drugbank.tsv'

In [3]:
df_mentions = pd.read_csv(mentions_file, sep='\t', header=0, index_col=False)
df_mentions.head()

,filename,label,start_span,end_span,text
0,casos_clinicos_cardiologia10,FARMACO,2562,2574,Cefazolina2g
1,casos_clinicos_cardiologia10,FARMACO,2587,2598,gentamicina
2,casos_clinicos_cardiologia10,FARMACO,2614,2625,rifampicina
3,casos_clinicos_cardiologia10,FARMACO,3229,3238,amikacina
4,casos_clinicos_cardiologia10,FARMACO,3244,3255,vancomicina


In [4]:
df_external_mentions = pd.read_csv(external_mentions_file, sep='\t', header=0, index_col=False)
df_external_mentions.head()

,file_name,label,start_index,end_index,drug_in_text,reference,original_drug,context
0,casos_clinicos_cardiologia10,FARMACO,2562,2572,Cefazolina,Arbol_Medicamentos_DSCA_Spanish-J01DB04-CIMAMe...,cefazolina,CONTEXT:al.Tratamiento:Cefazolina2g c/8 hs iv...
1,casos_clinicos_cardiologia10,FARMACO,2587,2598,gentamicina,Arbol_Medicamentos_DSCA_Spanish-D06AX07-J01GB0...,gentamicina,CONTEXT:ina2g c/8 hs iv; gentamicina 3mg/kg/d...
2,casos_clinicos_cardiologia10,FARMACO,2614,2625,rifampicina,Arbol_Medicamentos_DSCA_Spanish-J04AB02-CIMAMe...,rifampicina,CONTEXT:na 3mg/kg/día iv; rifampicina 600 mg c...
3,casos_clinicos_cardiologia10,FARMACO,3229,3238,amikacina,Arbol_Medicamentos_DSCA_Spanish-D06AX12-J01GB0...,amikacina,CONTEXT: Profilaxis preQ: amikacina 1g y vanco...
4,casos_clinicos_cardiologia10,FARMACO,3244,3255,vancomicina,Arbol_Medicamentos_DSCA_Spanish-A07AA09-J01XA0...,vancomicina,CONTEXT:Q: amikacina 1g y vancomicina 1g.• Rec...


In [5]:
# filter drug bank
# remove start/end whitespaces and adjust indices
for index, row in df_external_mentions.iterrows():
    try:
        start = int(row['start_index'])
        end  = int(row['end_index'])
    except:
        print(index, row)

In [6]:
df_external_mentions = df_external_mentions.astype({'start_index': 'int64', 'end_index': 'int64'})

In [7]:
df_external_mentions['drugbank'] = df_external_mentions['reference'].apply(lambda x: 'DrugBank' in str(x))

In [8]:
df_external_mentions_filter = df_external_mentions[~df_external_mentions['drugbank']]
df_external_mentions_filter.head()

,file_name,label,start_index,end_index,drug_in_text,reference,original_drug,context,drugbank
0,casos_clinicos_cardiologia10,FARMACO,2562,2572,Cefazolina,Arbol_Medicamentos_DSCA_Spanish-J01DB04-CIMAMe...,cefazolina,CONTEXT:al.Tratamiento:Cefazolina2g c/8 hs iv...,False
1,casos_clinicos_cardiologia10,FARMACO,2587,2598,gentamicina,Arbol_Medicamentos_DSCA_Spanish-D06AX07-J01GB0...,gentamicina,CONTEXT:ina2g c/8 hs iv; gentamicina 3mg/kg/d...,False
2,casos_clinicos_cardiologia10,FARMACO,2614,2625,rifampicina,Arbol_Medicamentos_DSCA_Spanish-J04AB02-CIMAMe...,rifampicina,CONTEXT:na 3mg/kg/día iv; rifampicina 600 mg c...,False
3,casos_clinicos_cardiologia10,FARMACO,3229,3238,amikacina,Arbol_Medicamentos_DSCA_Spanish-D06AX12-J01GB0...,amikacina,CONTEXT: Profilaxis preQ: amikacina 1g y vanco...,False
4,casos_clinicos_cardiologia10,FARMACO,3244,3255,vancomicina,Arbol_Medicamentos_DSCA_Spanish-A07AA09-J01XA0...,vancomicina,CONTEXT:Q: amikacina 1g y vancomicina 1g.• Rec...,False


In [9]:
df_external_mentions_filter.shape

(2879, 9)

In [10]:
df_external_mentions.shape

(3390, 9)

In [11]:
for index, row in df_external_mentions.iterrows():
    mention_text = str(row['drug_in_text'])
    left_whitespace = mention_text.lstrip()
    if len(mention_text) - len(left_whitespace) > 0:
        df_external_mentions.at[index, 'start_index'] = row['start_index'] + (len(mention_text) - len(left_whitespace))
        
    right_whitespace = mention_text.rstrip()
    if len(mention_text) - len(right_whitespace) > 0:
        df_external_mentions.at[index, 'end_index'] = row['end_index'] - (len(mention_text) - len(right_whitespace))
        
    if (len(mention_text) - len(left_whitespace) > 0) or (len(mention_text) - len(right_whitespace) > 0):
        df_external_mentions.at[index, 'drug_in_text'] = mention_text.strip()

In [12]:
def filter_spans(spans):
    get_sort_key = lambda span: (span['end_span'] - span['start_span'], -span['start_span'])
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens: Set[int] = set()
    for span in sorted_spans:
        # Check for end - 1 here because boundaries are inclusive
        if span['start_span'] not in seen_tokens and span['end_span'] - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span['start_span'], span['end_span']))
    result = sorted(result, key=lambda span: span['start_span'])
    return result

In [13]:
external_filenames = list(df_external_mentions['file_name'].values)
original_filenames = list(df_mentions['filename'].values)
original_filenames.extend(external_filenames)
filenames = list(set(original_filenames))
filenames.sort()

In [14]:
all_spans = []
for filename in filenames:
    # get both mentions
    df_mentions_file = df_mentions[df_mentions['filename'] == filename]
    #df_external_mentions_file = df_external_mentions[df_external_mentions['file_name'] == filename]
    df_external_mentions_file = df_external_mentions_filter[df_external_mentions_filter['file_name'] == filename]
    # merge spans
    spans = []

    for index, mention in df_mentions_file.iterrows():
        spans.append({
            'filename': mention['filename'],
            'label': mention['label'],
            'start_span': mention['start_span'],
            'end_span': mention['end_span'],
            'text': mention['text']
        })

    for index, mention in df_external_mentions_file.iterrows():
        spans.append({
            'filename': mention['file_name'],
            'label': mention['label'],
            'start_span': mention['start_index'],
            'end_span': mention['end_index'],
            'text': mention['drug_in_text']
        })

    filtered_spans = filter_spans(spans)
    all_spans.extend(filtered_spans)

In [15]:
df_all_mentions = pd.DataFrame.from_records(all_spans)
df_all_mentions.head()

,filename,label,start_span,end_span,text
0,casos_clinicos_cardiologia10,FARMACO,2562,2574,Cefazolina2g
1,casos_clinicos_cardiologia10,FARMACO,2587,2598,gentamicina
2,casos_clinicos_cardiologia10,FARMACO,2614,2625,rifampicina
3,casos_clinicos_cardiologia10,FARMACO,3229,3238,amikacina
4,casos_clinicos_cardiologia10,FARMACO,3244,3255,vancomicina


In [16]:
df_all_mentions.drop_duplicates(inplace=True)

In [17]:
df_all_mentions[['filename', 'label', 'start_span', 'end_span', 'text']].to_csv(result_file, sep='\t', index=False)

In [ ]:
# en - |0.8475|0.9076|0.8765 - combined w/ drugbank, |0.8732|0.9|0.8864 - w/o drugbank
# es - |0.6252|0.9092|0.7409 - combined w/ drugbank, |0.6281|0.9096|0.743 - w/o drugbank
# it - |0.5766|0.8623|0.6911 - combined w/ drugbank, |0.5815|0.8642|0.6952 - combined w/o drugbank